In [ ]:
#one-hot encoding
import numpy as np

samples = ['The cat sat on the mat.','The dog eat my homework.']

token_index = {}

# 建立字典
for sample in samples:
  for word in sample.split():
    if word not in token_index:
      token_index[word]=len(token_index)+1

# 字典建立完成

max_length = 10

results = np.zeros(shape=(len(samples),max_length,max(token_index.values())+1))

print(results.shape)

for i,sample in enumerate(smaples):
  for j , word in list(enumerate(sample.split()))[:max_length]:
    index = token_index.get(word)
    results[i,j,index] = 1. # 把對應元素設為1

results

In [ ]:
#字元的one-hot encoding
import string

samples = ['The cat sat on the mat.','The dog eat my homework.']
characters = string.printable
print(len(characters))

token_index = dict(zip(characters,range(1,len(characters)+1)))

# 字典建立完成

max_length = 50

results = np.zeros(shape=(len(samples),max_length,max(token_index.values())+1))

print(results.shape)

for i,sample in enumerate(smaples):
  for j , character in enumerate(sample):
    index = token_index.get(character)
    results[i,j,index] = 1. # 把對應元素設為1

results

In [ ]:
#用Keras的內建工具 做 one-hot encoding
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.','The dog eat my homework.']

tokenizer = Tokenizer(num_words=1000) # 建立分類器，處理前1000個最常用的單字
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)

print(sequences)

one_hot_results = tokenizer.texts_to_matrix(samples,mode='binary')

print(one_hot_results.shape)
print(one_hot_results[0][:15]) # 第一個樣本的前15個
print(one_hot_results[1][:15]) # 第二個樣本的前15個

word_index = tokenizer.word_index
print('Found %s unique tokens'% len(word_index))

In [ ]:
#使用雜湊技巧做 one-hot encoding
samples = ['The cat sat on the mat.','The dog eat my homework.']

dimensionality = 1000
max_length = 10

results = np.zeros((len(samples),max_length,dimensionality))

for i ,smaple in enumerate(samples):
  for j,eord in list(enumerate(sample.split()))[:max_length]:
    index = abs(hash(word))%dimensionality # 將token雜湊成0到1000之間的隨機整數索引
    results[i,j,index] = 1.

print(results.shape)

In [ ]:
#用Keras 的 Embeddung Layer來實作文字崁入法
from keras.layers import Embedding
embedding_layer = Embedding(1000,64)

In [ ]:
from keras.datasets import imdb
from keras import preprocessing

max_features = 10000
maxlen = 20

(x_train,y_train),(x_test,y_test) = imdb.load_data(num_words=max_features) # 將文字評論以整數(鍵值)list載入

print(x_train.shape)
x_train = preprocessing.sequence.pad_sequences(x_train,maxlen=maxlen)
print(x_train.shape)

print(x_train[0])

x_test = preprocessing.sequence.pad_sequences(x_test,maxlen = maxlen)

In [ ]:
#訓練模型
from keras.models import Sequential
from keras.layers import Flatten,Dense,Embedding

model = Sequential()
model.add(Embedding(10000,8,input_length=maxlen))
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
model.summary()

history = model.fit(x_train,y_train,epochs=10,batch_size=32,validation_split=0.2)
# 驗證準確度: 約 74.66%
print(history.history['val_acc'][-1])

In [ ]:
#自原始文字資料到文字崁入向量
import os

imdb_dir = './data/aclImdb'
train_dir=os.path.join(imdb_dir,'train')

labels=[]
texts=[]

for label_type in ['neg','pos']:
    dir_name = os.path.join(train_dir,label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name,fname),encoding='utf8')
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)
print(len(texts))

In [ ]:
#對資料進行向量化

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np 

maxlen = 100
training_smaples = 200
validation_samples = 10000
max_words = 10000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index

print('共使用了 %s 個token字詞' %len(word_index))

data = pad_sequences(sequences,maxlen=maxlen)
labels = np.asarray(labels)

print('資料張量 shape:',data.shape)
print('標籤張量 shape:',labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_smaples]
y_train = labels[:training_smaples]
x_val = data[training_smaples:training_smaples+validation_samples]
y_val = labels[training_smaples:training_smaples+validation_samples]

In [ ]:
#下載 GLOVE 文字崁入向量
#使用其中的100維數值向量做為所需要的資料

glove_dir = './data'

embeddings_index ={}
f = open(os.path.join(glove_dir,'glove.6B.100d.txt'),encoding='UTF-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('共使用了 %s 個文字崁入向量' % len(embeddings_index))

In [ ]:
#準備 GloVe 文字嵌入向量矩陣
embedding_dim= 100

embedding_matrix = np.zeros((max_words,embedding_dim))

for word,i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_matrix[i] is not None:
            embedding_matrix[i] = embedding_vector

In [ ]:
#模型定義
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
# 參數 樣本數, 嵌入向量維度, 
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False # 凍結崁入層

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))
model.save_weights('./model/pre_trained_glove_model.h5')

In [ ]:
#繪製結果

import matplotlib.pyplot as plt 

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1 , len(acc)+1)

plt.plot(epochs,acc,'bo',label='Training acc')
plt.plot(epochs,val_acc ,'b',label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs,loss,'bo',label='Training loss')
plt.plot(epochs,val_loss ,'b',label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
#訓練相同模型而不使用預先訓練的文字嵌入向量
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))


############### 繪製
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
test_dir = os.path.join(imdb_dir, 'test')

labels = []
texts = []

for label_type in ['neg', 'pos']:
	dir_name = os.path.join(test_dir, label_type)
	for fname in sorted(os.listdir(dir_name)):
		if fname[-4:] == '.txt':
			f = open(os.path.join(dir_name, fname), encoding='UTF-8')
			texts.append(f.read())
			f.close()
			if label_type == 'neg':
				labels.append(0)
			else:
				labels.append(1)

sequences = tokenizer.texts_to_sequences(texts)
x_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(labels)

model.load_weights('./model/pre_trained_glove_model.h5')
model.evaluate(x_test, y_test)

In [ ]:
#6-2 循環神經網路 RNN
#以 Numpy 實現簡單的 RNN
#概念：將輸入和狀態轉換成輸出→參數化兩個權重矩陣
import numpy as np

timesteps = 100   # 輸入序列資料中的時間點數量
input_features = 32  # 輸入特徵空間的維度數
output_features = 64  # 輸出特徵空間的維度數

inputs = np.random.random((timesteps, input_features))  # 輸入資料：隨機產生數值以便示範

state_t = np.zeros((output_features, ))  # 初始狀態：全零向量

W = np.random.random((output_features, input_features))  # 建立隨機權重矩陣
U = np.random.random((output_features, output_features))
b = np.random.random((output_features, ))

successive_outputs = []
for input_t in inputs:  #  input_t 是個向量, shape 為 (input_features, )
    output_t = np.tanh(np.dot(W, input_t) + np.dot(U, state_t) + b)  # 結合輸入與當前狀態(前一個輸出)以取得當前輸出
    successive_outputs.append(output_t)  # 將此輸出儲存在列表中
    state_t = output_t  #更新下一個時間點的網絡狀態

final_output_sequence = np.concatenate(successive_outputs, axis=0)  
print(final_output_sequence.shape)

In [ ]:
#透過Keras的SimpleRNN層來進行實作
from keras.models import Sequential
from keras.layers import Embedding , SimpleRNN

model = Sequential()
model.add(Embedding(10000,32))
model.add(SimpleRNN(32))
model.summary()

In [ ]:
model = Sequential()
model.add(Embedding(10000,32))
model.add(SimpleRNN(32,return_sequences=True)) # 增加代表時間點的軸
model.summary()

In [ ]:
#增加多個循環層可以增加神經網路的表現能力
model = Sequential()
model.add(Embedding(10000,32))
model.add(SimpleRNN(32,return_sequences=True)) 
model.add(SimpleRNN(32,return_sequences=True)) 
model.add(SimpleRNN(32,return_sequences=True)) 
model.add(SimpleRNN(32))
model.summary()

In [ ]:
#準備 IMDB 資料
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 10000  #考慮做為特徵的文字數量
maxlen = 500  # 我們只看每篇評論的前 500 個文字

batch_size = 32

print('讀取資料...')
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=max_features)
print(len(input_train), 'train sequences') # 25000 筆訓練用序列資料 (評論)
print(len(input_test), 'test sequences')	# 25000 筆測試用序列資料

print('Pad sequences (samples x time)')
input_train = sequence.pad_sequences(input_train, maxlen=maxlen) # 1. 只看每篇評論的前 500 個文字, 多的去除, 不足填補
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)
print('input_train shape:', input_train.shape)	# shape=(25000, 500)
print('input_test shape:', input_test.shape)	# shape=(25000, 500)

In [ ]:
#以嵌入向量 Embedding 層和 SimpleRNN 層訓練模型
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding, SimpleRNN

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(input_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

In [ ]:
#繪製結果
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs,acc,'bo',label='Training acc')
plt.plot(epochs,val_acc ,'b',label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs,loss,'bo',label='Training loss')
plt.plot(epochs,val_loss ,'b',label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
#長期短期記憶(Long Short-Term Memory,LSTM)
#在 Keras 中使用 LSTM 層
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['acc'])
history = model.fit(input_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')

In [ ]:
#6-3 循環神經網路的進階用法¶
#循環丟棄 (Recurrent dropout)
#堆疊循環層 (Stacking recurrent layers)
#雙向循環層 (Bidirectional recurent layers)

#檢視耶拿天氣資料集的資料
import os

data_dir = r'./data/'  # 您的 jena_climate 資料夾路徑
fname = os.path.join(data_dir, 'jena_climate_2009_2016.csv') # 資料集完整路徑

f = open(fname)
data = f.read()
f.close()

lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]

print(header)  # 
print(len(header))
print(len(lines)

In [ ]:
#解析資料
import numpy as np

float_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(',')[1:]]
    float_data[i, :] = values
print(float_data.shape)   # 共有 420551 個時間點的天氣資料, 每個包含 14 種天氣數值

In [ ]:
#觀察整個年度的變化
from matplotlib import pyplot as plt

temp = float_data[:, 1] # 索引 1 為 temperature 資料
plt.plot(range(len(temp)), temp)
plt.show()

In [ ]:
#觀察資料集中前10天的溫度資料
plt.plot(range(1440), temp[:1440])
plt.show()

In [ ]:
#標準化資料
mean = float_data[:200000].mean(axis=0)
float_data -= mean
std = float_data[:200000].std(axis=0)
float_data /= std

In [ ]:
#定義產生器函式以產生時間序列樣本及其目標樣本
def generator(data,lookback,delay,min_index,max_index,shuffle=False,batch_size=128,step=6):
    if max_index is not None:
        max_index = len(data) - delay -1
    
    i = min_index + lookback

    while 1:
        if shuffle:
            rows = np.random.randint(min_index+lookback , max_index ,size=batch_size)
        else:
            if i+batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i,min(i+batch_size,max_index))
            i += len(rows)
            
        samples = np.zeros((len(rows),lookback//step,data.shape[-1]))

        targets = np.zeros((len(rows),))

        for j,row in enumerate(rows):
            indices = range(rows[j]-lookback,rows[j],step)
            samples[j]= data[indices]
            targets[j] = data[rows[j]+delay][1]
        
        yield samples,targets

In [ ]:
#建立訓練資料、驗證資料與測試資料產生器
lookback = 1440
step = 6
delay = 144
batch_size = 128

# 訓練資料產生器
train_gen = generator(float_data,lookback=lookback,delay=delay,min_index = 0,max_index=200000,shuffle=True,step=step,batch_size=batch_size)

# 驗證資料產生器
val_gen = generator(float_data,lookback=lookback,delay=delay,min_index = 200001,max_index=300000,shuffle=True,step=step,batch_size=batch_size)

# 測試資料產生器
val_gen = generator(float_data,lookback=lookback,delay=delay,min_index = 300001,max_index=None,shuffle=True,step=step,batch_size=batch_size)

val_steps = (300000-200001-lookback) // batch_size
test_steps = (len(float_data)-300001-lookback) // batch_size

In [ ]:
def evaluate_naive_method():
    batch_maes = []
    for step in range(val_steps):           # 計算所有驗證集資料
        samples , targets = next(val_gen)   # 驅動產生器
       # print(samples.shape)                # 回朔1440個時間點，並以6個時間點作為間隔進行取樣→共產生1440/6，讓他吐出一個批次輛
       # print(targets.shape)
        preds = samples[:,-1,1]
        mae = np.mean(np.abs(preds-targets))
        batch_maes.append(mae)
    print(np.mean(batch_maes))

evaluate_naive_method()

In [ ]:
celsius_mae = 0.29*std[1]
celsius_mae #得到攝氏溫度誤差2.57℃